# Advanced Certification Program in Computational Data Science
## A program by IISc and TalentSprint
### Mini-Project: Implementation of Multiple Linear Regression using MPI and OpenMP

## Learning Objectives

At the end of the mini-project, you will be able to :

* understand the collective communication operations like scatter, gather, broadcast
* understand the blocking and non-blocking communication
* implement multiple linear regression and run it using MPI
* implement the multiple linear regression based predictions using OpenMP

### Dataset

The dataset chosen for this mini-project is [Combined Cycle Power Plant](https://archive.ics.uci.edu/ml/datasets/combined+cycle+power+plant). The dataset is made up of 9568 records and 5 columns. Each record contains the values for Ambient Temperature, Exhaust Vaccum, Ambient Pressure, Relative Humidity and Energy Output.

Predicting full load electrical power output of a base load power plant is important in order to maximize the profit from the available megawatt hours.  The base load operation of a power plant is influenced by four main parameters, which are used as input variables in the dataset, such as ambient temperature, atmospheric pressure, relative humidity, and exhaust steam pressure. These parameters affect electrical power output, which is considered as the target variable.

**Note:** The data was collected over a six year period (2006-11).

## Information

#### MPI in a Nutshell

MPI stands for "Message Passing Interface". It is a library of functions (in C / Python) or subroutines (in Fortran) that you insert into source code to perform data communication between processes. MPI was developed over two years of discussions led by the MPI Forum, a group of roughly sixty people representing some forty organizations.

To know more about MPI click [here](https://hpc-tutorials.llnl.gov/mpi/)


#### Multiple Linear Regression

Multiple regression is an extension of simple linear regression. It is used when we want to predict the value of a variable based on the value of two or more other variables. The variable we want to predict is called the dependent variable (or sometimes, the outcome, target or criterion variable). The variables we are using to predict the value of the dependent variable are called the independent variables (or sometimes, the predictor, explanatory or regressor variables).

**Note:** We will be using the mpi4py Python package for MPI based code implementation

## Grading = 20 Points

**Run the below code to install mpi4py package**

In [ ]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.3/466.3 kB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.3-cp311-cp311-linux_x86_64.whl size=4438170 sha256=2530967a4b16259d3d4fda345c3a0be4f30a976bed2e7240e6e18efb0a20a54c
  Stored in directory: /root/.cache/pip/wheels/5c/56/17/bf6ba37aa971a191a8b9eaa188bf5ec855b8911c1c56fb1f84
Successfully built mpi4py


#### Importing Necessary Packages

In [ ]:
# Importing pandas
import pandas as pd
# Importing Numpy
import numpy as np
# Importing MPI from mpi4py package
from mpi4py import MPI
# Importing sqrt function from the Math
from math import sqrt
# Importing Decimal, ROUND_HALF_UP functions from the decimal package
from decimal import Decimal, ROUND_HALF_UP
from sklearn.preprocessing import StandardScaler
import time

#### Downloading the data

In [ ]:
#@title Download the data
!wget -qq https://cdn.iisc.talentsprint.com/CDS/Datasets/PowerPlantData.csv

### Overview

* Load the data and perform data pre-processing
* Identify the features, target and split the data into train and test
* Implement multiple Linear Regression by estimating the coefficients on the given data
* Use MPI package to distribute the data and implement `communicator`
* Define functions for each objective and make a script (.py) file to execute using MPI command
* Use OpenMP component to predict the data and calculate the error on the predicted data
* Implement the Linear Regression from `sklearn` and compare the results

#### Exercise 1: Load data (1 point)

Write a function that takes the filename as input and loads the data in a pandas dataframe with the column names as Ambient Temperature, Exhaust Vaccum, Ambient Pressure, Relative Humidity and Energy Output respectively.

**Hint:** read_csv()


In [ ]:
FILENAME = "/content/PowerPlantData.csv" # File path

# YOUR CODE HERE to Define a function to load the data
df = pd.read_csv(FILENAME)

#### Exercise 2: Explore data (1 point)

Write a function that takes the data loaded using the above defined function as input and explore it.

**Hint:** You can define and check for following things in the dataset inside a function

- checking for the number of rows and columns
- summary of the dataset
- check for the null values
- check for the duplicate values

In [ ]:
# YOUR CODE HERE
print(df.shape)
print(df.info())
print(df.describe())
print(df.isnull().sum())
print(len(df[df.duplicated()]))

(9568, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB
None
                AT            V           AP           RH           PE
count  9568.000000  9568.000000  9568.000000  9568.000000  9568.000000
mean     19.651231    54.305804  1013.259078    73.308978   454.365009
std       7.452473    12.707893     5.938784    14.600269    17.066995
min       1.810000    25.360000   992.890000    25.560000   420.260000
25%      13.510000    41.740000  1009.100000    63.327500   439.750000
50%      20.345000    52.080000  1012.940000    74.975000   451.550000
75%      25.720000    66.540000  1017.260000    84.830000   468.430000
max      3

#### Exercise 3: Handle missing data (1 point)

After exploring the dataset if there are any null values present in the dataset then define a function that takes data loaded using the above defined function as input and handle the null values accordingly.

**Hint:**

- Drop the records containing the null values - dropna()
- Replace the null values with the mean/median/mode - fillna()

In [ ]:
# Function to handle missing data

# YOUR CODE HERE

#### Exercise 4: Scale the data (1 point)

Write a function that takes the data after handling the missing data as input and returns the standardized data.

**Hint:**

- standardization of the data  can be performed using the below formula

$ (x - mean(x)) / std(x) $

In [ ]:
# Defining a function to standardize the data

# YOUR CODE HERE
def scale_data(df):

    scaler = StandardScaler()
    scaler.fit(df)
    scaled_data = scaler.transform(df)

    return(pd.DataFrame(scaled_data, columns=df.columns, index = df.index))

#### Exercise 5: Feature selection (1 point)

Write a function that takes scaled data as input and returns the features and target variable values

**Hint:**

- Features: AmbientTemperature, ExhaustVaccum, AmbientPressure, RelativeHumidity
- Target Variable: EnergyOutput

In [ ]:
# Define a function

# YOUR CODE HERE
def split_data(df):

    feature_cols = ['AT','V','AP','RH']
    target_col = ['PE']

    features = df[feature_cols]
    target = df[target_col]

    return features,target

#### Exercise 6: Correlation (1 point)

Calculate correlation between the variables

In [ ]:
# YOUR CODE HERE
def calculate_corelation(df):
    return df.corr()

In [ ]:
corr_matrix = calculate_corelation(scale_data(df))
print(corr_matrix)

          AT         V        AP        RH        PE
AT  1.000000  0.844107 -0.507549 -0.542535 -0.948128
V   0.844107  1.000000 -0.413502 -0.312187 -0.869780
AP -0.507549 -0.413502  1.000000  0.099574  0.518429
RH -0.542535 -0.312187  0.099574  1.000000  0.389794
PE -0.948128 -0.869780  0.518429  0.389794  1.000000


#### Exercise 7: Estimate the coefficients (2 points)

Write a function that takes features and target as input and returns the estimated coefficient values

**Hint:**

- Calculate the estimated coefficients using the below formula

$ β = (X^T X)^{-1} X^T y $

- transpose(), np.linalg.inv()

In [ ]:
# Calculating the coeffients

# YOUR CODE HERE
def calculate_coff(features,target):
    X = features.to_numpy()
    y = target.to_numpy()

    X_transpose = X.T
    coeff = np.linalg.inv(X_transpose @ X) @ X_transpose @ y
    return coeff

In [ ]:
features, target = split_data(scale_data(df))
coeff = calculate_coff(features, target)
print(coeff)

[[-0.86350078]
 [-0.17417154]
 [ 0.02160293]
 [-0.13521023]]


#### Exercise 8: Fit the data to estimate the coefficients (2 points)

Write a function named fit which takes features and targets as input and returns the intercept and coefficient values.

**Hint:**

- create a dummy column in the features dataframe which is made up of all ones
- convert the features dataframe into numpy array
- call the estimated coefficients function which is defined above
- np.ones(), np.concatenate()

In [ ]:
# defining a fit function
def fit(x, y):
    # YOUR CODE HERE
    x['Dummy'] = 1
    print(x.head(1))
    coeff = calculate_coff(x,y)
    return coeff


In [ ]:
coeff = fit(features, target)
print(coeff[:4])

         AT         V        AP        RH  Dummy
0 -1.517862 -1.065205 -0.407357  1.143944      1
[[-0.86350078]
 [-0.17417154]
 [ 0.02160293]
 [-0.13521023]]


#### Exercise 9: Predict the data on estimated coefficients (1 point)

Write a function named predict which takes features, intercept and coefficient values as input and returns the predicted values.

**Hint:**

- Fit the intercept, coefficients values in the below equation

  $y = b_0 + b_1*x + ... + b_i*x_i$

In [ ]:
 # fucntion to predict the values
def predict(x, intercept, coefficients):
    '''
    y = b_0 + b_1*x + ... + b_i*x_i
    '''
    #YOUR CODE HERE
    coefficients = coefficients.flatten()
    intercept = intercept.flatten()

    predictions = np.zeros(x.shape[0])

    for index, row in x.reset_index(drop=True).iterrows():
            predictions[index] = intercept + np.dot(coefficients, row.iloc[:len(coefficients)])

    predictions_df = pd.DataFrame(predictions,columns=['PE_Pred'])
    return predictions_df

In [ ]:
prediction = predict(features,coeff[4],coeff[:4])
print(prediction)

<ipython-input-15-39becd870d04>:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions[index] = intercept + np.dot(coefficients, row.iloc[:len(coefficients)])


       PE_Pred
0     1.332730
1    -0.534559
2    -0.936009
3     0.076408
4     0.889608
...        ...
9563  0.597073
9564 -1.803272
9565  0.528409
9566 -0.022664
9567 -0.411558

[9568 rows x 1 columns]


#### Exercise 10: Root mean squared error (1 point)

Write a function to calculate the RMSE error.

**Hint:**

- [How to calculate the RSME error](https://towardsdatascience.com/what-does-rmse-really-mean-806b65f2e48e)

In [ ]:
# Define a function to calculate the error

# YOUR CODE HERE
def calculate_rsme(target,prediction):
    target = target.to_numpy()
    prediction = prediction.to_numpy()

    squared_errors = (prediction - target) ** 2
    mean_squared_error = np.mean(squared_errors)

    rmse_value = np.sqrt(mean_squared_error)

    return rmse_value

In [ ]:
rsme = calculate_rsme(target,prediction)
print(rsme)

0.26702792024008715


#### Exercise 11: Split the data into train and test (1 point)

Write a function named train_test_split which takes features and targets as input and returns the train and test sets respectively.

**Hint:**

- Shuffle the data
- Consider 70 % of data as a train set and the rest of the data as a test set

In [ ]:
# YOUR CODE HERE
def train_test_split(X):
    X = X.sample(frac=1, random_state=42).reset_index(drop=True)
    num_rows = int(0.7 * len(X))
    remain_rows = len(X) - num_rows
    train = X.iloc[:num_rows, :].copy()
    test = X.tail(remain_rows).copy()
    return train, test

In [ ]:
train,test = train_test_split(scale_data(df))
print(train.shape,test.shape)
print(train.head(1))

(6697, 5) (2871, 5)
         AT         V        AP        RH        PE
0  1.348451  0.239564 -1.284687 -1.093118 -1.236077


In [ ]:
prediction = predict(test,coeff[4],coeff[:4])
print(prediction)

       PE_Pred
0    -0.512701
1    -1.607746
2    -0.997989
3     1.447719
4     1.348577
...        ...
2866 -0.843814
2867 -1.121608
2868  0.455895
2869 -0.968580
2870  1.658825

[2871 rows x 1 columns]


<ipython-input-55-8c0aff652542>:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions[index] = intercept + np.dot(coefficients, row.iloc[:len(coefficients)])


#### Exercise 12: Implement predict using OpenMP (1 point)

Get the predictions for test data and calculate the test error(RMSE) by implementing the OpenMP (pymp)

**Hints:**

* Using the pymp.Parallel implement the predict function (use from above)

* Call the predict function by passing test data as an argument

* calculate the error (RMSE) by comparing the Actual test data and predicted test data

In [ ]:
!pip install pymp-pypi

  Preparing metadata (setup.py) ... done
  Created wheel for pymp-pypi: filename=pymp_pypi-0.5.0-py3-none-any.whl size=10314 sha256=b00f491c7c2c3fd6b57777e5ee51ebadcee0446955c900e08d7aa37e91ce711c
  Stored in directory: /root/.cache/pip/wheels/3d/6c/ef/356575a818508a9c37c4513a5139063d522ea002bcebd15bbe
Successfully built pymp-pypi


In [ ]:
import pymp
# YOUR CODE HERE

def predict_parallel(x, intercept, coefficients):
    '''
    y = b_0 + b_1*x + ... + b_i*x_i
    '''
    #YOUR CODE HERE
    coefficients = coefficients.flatten()
    intercept = intercept.flatten()

    num_rows = x.shape[0]
    predictions = pymp.shared.array((num_rows,),dtype='float64')

    with pymp.Parallel(4) as p:
        for index in p.range(0, num_rows):
            row = x.iloc[index]
            predictions[index] = (intercept + np.dot(coefficients, row.iloc[:len(coefficients)])).item()

    predictions_df = pd.DataFrame(predictions,columns=['PE_Pred'])
    return predictions_df


In [ ]:
test_feature, test_target = split_data(test)
pred = predict_parallel(test_feature,coeff[4],coeff[:4])
print(calculate_rsme(test_target,pred))

0.2650547913407178


#### Exercise 13: Create a communicator (1 point)

Create a comunicator and define the rank and size

In [ ]:
# YOUR CODE HERE
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

#### Exercise 14: Divide the data into slices (1 point)

Write a function named dividing_data which takes train features set, train target set, and size of workers as inputs and returns the sliced data for each worker.

![img](https://cdn.iisc.talentsprint.com/CDS/Images/MiniProject_MPI_DataSlice.JPG)

For Example, if there are 4 processes, slice the data into 4 equal parts with 25% ratio

**Hint:**

- Divide the Data equally among the workers
  - Create an empty list
  - Iterate over the size of workers
  - Append each slice of data to the list

In [ ]:
def dividing_data(x_train, y_train, size_of_workers):
    # Size of the slice
    worker_data = []

    slice_for_each_worker = int(Decimal(x_train.shape[0]/size_of_workers).quantize(Decimal('1.'), rounding = ROUND_HALF_UP))
    print('Slice of data for each worker: {}'.format(slice_for_each_worker))
    # YOUR CODE HERE

    row_num = 0
    i = 0
    for i in range(size_of_workers):

        end_index = row_num+slice_for_each_worker
        end_index = min(end_index, len(x_train))
        print(end_index)

        worker_data.append((x_train.iloc[row_num:end_index, :].copy(),
                            y_train.iloc[row_num:end_index, :].copy()))
        row_num = end_index

    return worker_data



In [ ]:
data = dividing_data(test_feature, test_target,4)
data[0][1]

Slice of data for each worker: 718
718
1436
2154
2871


,PE
6697,-0.804812
6698,-1.090759
6699,-0.767311
6700,1.687848
6701,0.910283
...,...
7410,1.731209
7411,1.269475
7412,1.386080
7413,0.338390


#### Exercise 15: Prepare the data in root worker to assign data for all the workers (1 point)

- When it is the root worker, perform the below operation:
    - Store the features and target values in separate variables
    - Split the data into train and test sets using the train_test_split function defined above
    - Divide the data among the workers using the dividing_data function above

In [ ]:
# YOUR CODE HERE

if rank == 0:
    features, target = split_data(scale_data(df))
    X_train,X_test = train_test_split(features)
    y_train,y_test = train_test_split(target)
    worker_data = dividing_data(X_train, y_train, 4)


#### Exercise 16: Scatter and gather the data (1 point)

Perform the below operations:

- Send slices of the training set(the features data X and the expected target data Y) to every worker including the root worker
    - **Hint:** scatter()
    - use `barrier()` to block workers until all workers in the group reach a Barrier, to scatter from root worker.
- Every worker should get the predicted target Y(yhat) for each slice
- Get the new coefficient of each instance in a slice
    - **Hint:** fit function defined above
- Gather the new coefficient from each worker
    - **Hint:** gather()
    - Take the mean of the gathered coefficients
- Calculate the root mean square error for the test set

To know more about `scatter`, `gather` and `barrier` click [here](https://nyu-cds.github.io/python-mpi/05-collectives/)

In [ ]:
# YOUR CODE HERE
recv_data = comm.scatter(worker_data,root=0)
comm.barrier()

worker_coeff = fit(recv_data[0],recv_data[1])
recv_buff = None
recv_buff = comm.gather(worker_coeff,root=0)

if rank == 0:
    print(recv_buff)







#### Exercise 17: Make a script and execute everything in one place (1 point)

Write a script(.py) file which contains the code of all the above exercises in it so that you can run the code on multiple processes using MPI.

**Hint:**

- magic commands
- put MPI related code under main function
- !mpirun --allow-run-as-root -np 4 python filename.py

In [ ]:
# YOUR CODE HERE for scipt(.py)
# Importing pandas
import pandas as pd
# Importing Numpy
import numpy as np
# Importing MPI from mpi4py package
from mpi4py import MPI
# Importing sqrt function from the Math
from math import sqrt
# Importing Decimal, ROUND_HALF_UP functions from the decimal package
from decimal import Decimal, ROUND_HALF_UP
from sklearn.preprocessing import StandardScaler
import time

FILENAME = "PowerPlantData.csv" # File path

# YOUR CODE HERE to Define a function to load the data
df = pd.read_csv(FILENAME)

def scale_data(df):

    scaler = StandardScaler()
    scaler.fit(df)
    scaled_data = scaler.transform(df)

    return(pd.DataFrame(scaled_data, columns=df.columns, index = df.index))

def split_data(df):

    feature_cols = ['AT','V','AP','RH']
    target_col = ['PE']

    features = df[feature_cols]
    target = df[target_col]

    return features,target

def calculate_corelation(df):
    return df.corr()

def calculate_coff(features,target):
    X = features.to_numpy()
    y = target.to_numpy()

    X_transpose = X.T
    coeff = np.linalg.inv(X_transpose @ X) @ X_transpose @ y
    return coeff

def fit(x, y):
    # YOUR CODE HERE
    x['Dummy'] = 1
    coeff = calculate_coff(x,y)
    return coeff

def predict(x, intercept, coefficients):
    '''
    y = b_0 + b_1*x + ... + b_i*x_i
    '''
    #YOUR CODE HERE
    coefficients = coefficients.flatten()
    intercept = intercept.flatten()

    predictions = np.zeros(x.shape[0])
    print(x.shape)

    for index, row in x.reset_index(drop=True).iterrows():
            predictions[index] = intercept + np.dot(coefficients, row.iloc[:len(coefficients)])

    predictions_df = pd.DataFrame(predictions,columns=['PE_Pred'])
    return predictions_df

def calculate_rmse(target,prediction):
    target = target.to_numpy()
    prediction = prediction.to_numpy()

    squared_errors = (prediction - target) ** 2
    mean_squared_error = np.mean(squared_errors)

    rmse_value = np.sqrt(mean_squared_error)

    return rmse_value

def train_test_split(X):
    X = X.sample(frac=1, random_state=42).reset_index(drop=True)
    num_rows = int(0.7 * len(X))
    remain_rows = len(X) - num_rows
    train = X.iloc[:num_rows, :].copy()
    test = X.tail(remain_rows).copy()
    return train, test

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

def dividing_data(x_train, y_train, size_of_workers):
    # Size of the slice
    worker_data = []

    slice_for_each_worker = int(Decimal(x_train.shape[0]/size_of_workers).quantize(Decimal('1.'), rounding = ROUND_HALF_UP))
    # YOUR CODE HERE

    row_num = 0
    i = 0
    for i in range(size_of_workers):

        end_index = row_num+slice_for_each_worker
        end_index = min(end_index, len(x_train))

        worker_data.append((x_train.iloc[row_num:end_index, :].copy(),
                            y_train.iloc[row_num:end_index, :].copy()))
        row_num = end_index

    return worker_data

worker_data = None
if rank == 0:
    features, target = split_data(scale_data(df))
    X_train,X_test = train_test_split(features)
    y_train,y_test = train_test_split(target)
    worker_data = dividing_data(X_train, y_train, 4)

recv_data = comm.scatter(worker_data ,root=0)
comm.barrier()

worker_coeff = fit(recv_data[0],recv_data[1])

recv_buff = None
recv_buff = comm.gather(worker_coeff,root=0)

if rank == 0:
    all_coeffs = np.array(recv_buff)
    mean_coeffs = np.mean(all_coeffs, axis=0)
    print(mean_coeffs)
    y_pred = predict(X_test,mean_coeffs[4],mean_coeffs[:4])
    rmse_value = calculate_rmse(y_test,y_pred)
    print(rmse_value)


In [ ]:
# YOUR CODE HERE for MPI command
!mpirun --allow-run-as-root -np 4 python mpi.py 2> error.log

#### Exercise 18: Use Sklearn to compare (1 point)

Apply the Linear regression on the given data using sklearn package and compare with the above results

**Hint:**
* Split the data into train and test
* Fit the train data and predict the test data using `sklearn Linear Regression`
* Compare the coefficients and intercept with above estimated coefficients
* calculate loss (RMSE) on test data and predictions and compare

In [ ]:
# YOUR CODE HERE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming 'df' is your original DataFrame

# 1. Split the data into train and test
features, target = split_data(scale_data(df))  # Using your existing functions
X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.3, random_state=42
)

# 2. Fit the train data using sklearn Linear Regression
sklearn_model = LinearRegression()
sklearn_model.fit(X_train, y_train)

# 3. Predict the test data
sklearn_predictions = sklearn_model.predict(X_test)

# 4. Compare coefficients and intercept
print("Sklearn Coefficients:", sklearn_model.coef_)
print("Custom Coefficients:", coeff[:4])  # Assuming 'coeff' is from your custom fit
print("Sklearn Intercept:", sklearn_model.intercept_)
print("Custom Intercept:", coeff[4])  # Assuming 'coeff' is from your custom fit

# 5. Calculate and compare RMSE
sklearn_rmse = np.sqrt(mean_squared_error(y_test, sklearn_predictions))
#custom_rmse = calculate_rsme(y_test, prediction(X_test, coeff[4], coeff[:4]))
# Using your existing predict and calculate_rsme functions

print("Sklearn RMSE:", sklearn_rmse)
#print("Custom RMSE:", custom_rmse)

Sklearn Coefficients: [[-0.86659727 -0.17234908  0.01947262 -0.13665087]]
Custom Coefficients: [[-0.86350078]
 [-0.17417154]
 [ 0.02160293]
 [-0.13521023]]
Sklearn Intercept: [0.00029321]
Custom Intercept: [-1.58120045e-15]
Sklearn RMSE: 0.2620609279565783
